In [1]:
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F
import torch.optim as optim

import sys
sys.path.append('../')

from src import Dataset, EdgeDistributionDataset
from src.network import MLP

### Loading dataset

In [2]:
dataset = Dataset('/Users/mehdiazabou/Documents/Nerds/cell_type/data', force_process=False)
aggr_dict = {'e23Cux2': 'e23', 'i5Sst': 'i5Sst', 'i5Htr3a': 'i5Htr3a', 'e4Scnn1a': 'e4', 'e4Rorb': 'e4',
             'e4other': 'e4', 'e4Nr5a1': 'e4', 'i6Htr3a': 'i6Htr3a', 'i6Sst': 'i6Sst', 'e6Ntsr1': 'e6',
             'i23Pvalb': 'i23Pvalb', 'i23Htr3a': 'i23Htr3a', 'i1Htr3a': 'i1Htr3a', 'i4Sst': 'i4Sst', 'e5Rbp4': 'e5',
             'e5noRbp4': 'e5', 'i23Sst': 'i23Sst', 'i4Htr3a': 'i4Htr3a', 'i6Pvalb': 'i6Pvalb', 'i5Pvalb': 'i5Pvalb',
             'i4Pvalb': 'i4Pvalb'}
dataset.aggregate_cell_classes(aggr_dict)
# Split into train/val/test sets
dataset.split_cell_train_val_test(test_size=0.2, val_size=0.2, seed=1234)
dataset.split_trial_train_val_test(test_size=0.2, val_size=0.2, temp=True, seed=1234)

# bining
dataset.set_bining_parameters(bin_size=0.1) # in seconds, so this is 10ms

Found processed pickle. Loading from '/Users/mehdiazabou/Documents/Nerds/cell_type/data/processed/dataset.pkl'.


In [7]:
# Create Pytorch datasets
train_dataset = EdgeDistributionDataset(dataset, edge_dist_num_bins=10, mode='train', sampler='U20')
# fix population for validation set and test set (they will be different of course)
val_dataset = EdgeDistributionDataset(dataset, edge_dist_num_bins=10, mode='val', sampler='U20', cell_random_seed=1)
test_dataset = EdgeDistributionDataset(dataset, edge_dist_num_bins=10, mode='test', sampler='U20', cell_random_seed=1)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=5, collate_fn=train_dataset.collate_fn)
val_loader = DataLoader(val_dataset, batch_size=1, collate_fn=train_dataset.collate_fn)
test_loader = DataLoader(test_dataset, batch_size=1, collate_fn=train_dataset.collate_fn)

In [8]:
def train(model, device, train_loader, optimizer, epoch, log_interval=None):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if log_interval and batch_idx % log_interval == 0:
            print('Train Epoch: {} [({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, 100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, loader, tag):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            total += len(target)

    print('{} set: Accuracy: {}/{} ({:.0f}%)'.format(tag,
        correct, total,
        100. * correct / total))

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MLP(input_dims=train_dataset.num_bins, n_hiddens=[20, 20, 20], n_class=17, dropout_p=0.2).to(device)
print(model)

lr = 1e-2
optimizer = optim.Adam(model.parameters(), lr=lr)

epochs=100
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval=5)
    test(model, device, train_loader, 'Train')
    test(model, device, val_loader, 'Val')
    test(model, device, test_loader, 'Test')
    print('\n')

MLP(
  (model): Sequential(
    (fc1): Linear(in_features=10, out_features=20, bias=True)
    (relu1): ReLU()
    (drop1): Dropout(p=0.2, inplace=False)
    (fc2): Linear(in_features=20, out_features=20, bias=True)
    (relu2): ReLU()
    (drop2): Dropout(p=0.2, inplace=False)
    (fc3): Linear(in_features=20, out_features=20, bias=True)
    (relu3): ReLU()
    (drop3): Dropout(p=0.2, inplace=False)
    (out): Linear(in_features=20, out_features=17, bias=True)
  )
)
Train Epoch: 1 [(0%)]	Loss: 5.885549
Train Epoch: 1 [(42%)]	Loss: 2.984617
Train Epoch: 1 [(83%)]	Loss: 2.844153
Train set: Accuracy: 1405/20400 (7%)
Val set: Accuracy: 487/6800 (7%)
Test set: Accuracy: 450/6800 (7%)


Train Epoch: 2 [(0%)]	Loss: 2.840449
Train Epoch: 2 [(42%)]	Loss: 2.836102
Train Epoch: 2 [(83%)]	Loss: 2.825511
Train set: Accuracy: 1299/20400 (6%)
Val set: Accuracy: 425/6800 (6%)
Test set: Accuracy: 420/6800 (6%)


Train Epoch: 3 [(0%)]	Loss: 2.818066
Train Epoch: 3 [(42%)]	Loss: 2.812483
Train Epoch: 3 [

KeyboardInterrupt: 